## Running two ML models at the same time
This notebook is an example how to run two models side-by-side and combine results of both models. A video stream from a local camera is processed by the hand and face detection models. Combined result is then displayed.
OpenCV is required to run this sample.

This script works with 

0. DeGirum Cloud Server, 
1. AI server equipped with DeGirum ORCA accelerator shared via Peer-to-Peer VPN, 
2. AI server equipped with DeGirum ORCA accelerator running in local network and 
3. AI server equipped with DeGirum ORCA accelerator running on the same machine as this code. 

To try different options, you just need to change the __model_zoo_option_index__ in the code below. 

The script needs a web camera connected to the machine running this code. The __camera_index__ also needs to be specified in the code below.

### Specify your model zoo option and camera index here

In [ ]:
model_zoo_option_list = ['DEGIRUM_CLOUD_SERVER', 'P2P_VPN_SERVER', 'LOCAL_NETWORK_SERVER', 'LOCALHOST']
model_zoo_option_index=1
camera_index=1

### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg # import DeGirum PySDK
import cv2 # OpenCV
import os
from dotenv import load_dotenv

In [ ]:
# Some helper functions

def show(img, capt = "<image>"):
    # show opencv image
    cv2.imshow(capt, img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('x') or key == ord('q'):
        raise KeyboardInterrupt
    

In [ ]:
envs_loaded=load_dotenv()
model_zoo_option=model_zoo_option_list[model_zoo_option_index]
if model_zoo_option=='DEGIRUM_CLOUD_SERVER':
    zoo = dg.connect_model_zoo(os.getenv('DEGIRUM_CLOUD_SERVER_ADDRESS'), token=os.getenv('DEGIRUM_CLOUD_TOKEN'))
elif model_zoo_option=='P2P_VPN_SERVER':
    zoo=dg.connect_model_zoo(os.getenv('P2P_VPN_SERVER_ADDRESS'))
elif model_zoo_option=='LOCAL_NETWORK_SERVER':
    zoo=dg.connect_model_zoo(os.getenv('LOCAL_NETWORK_SERVER_ADDRESS'))
elif model_zoo_option=='LOCALHOST':
    zoo=dg.connect_model_zoo(os.getenv('LOCAL_HOST_ADDRESS'))
else:
    print('Model zoo option not supported \n')                   

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
hand_det_model = zoo.load_model("yolo_v5s_hand_det--512x512_quant_n2x_orca_1")
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")

# select OpenCV backend: needed to have overlay image in OpenCV format
hand_det_model.image_backend = 'opencv'
hand_det_model._model_parameters.InputImgFmt=['JPEG']
hand_det_model.input_numpy_colorspace='BGR'

face_det_model.image_backend = 'opencv' 
face_det_model._model_parameters.InputImgFmt=['JPEG']
face_det_model.input_numpy_colorspace='BGR'

In [ ]:
# open video stream from local camera 
stream = cv2.VideoCapture(camera_index)
if (stream.isOpened()== False):
    print("Error opening video stream")
else:
    print("Succesful in opening video stream")

In [ ]:
# define iterator function, which returns frames from camera 
def source(idx):
    # idx is the index of a buffer
    N = 2 # number of buffers
    bufs = [[]] * N
    assert idx < N
    while True:
        if len(bufs[idx]) == 0: # this buffer is empty: get frame from camera and add to all buffers
            ret, frame = stream.read()
            for s in bufs:
                s.insert(0, frame)
        yield bufs[idx].pop()

In [ ]:
try:
    # run person detection model on a camera stream
    for hands, faces in zip(hand_det_model.predict_batch(source(0)), face_det_model.predict_batch(source(1))):

        hands._inference_results += faces._inference_results
        show(hands.image_overlay, "Hands and Faces")

except KeyboardInterrupt:
    pass # ignore KeyboardInterrupt errors
finally:
    cv2.destroyAllWindows() # close OpenCV windows
stream.release() # release camera stream